In [ ]:
!pip install ultralytics torchvision kagglehub pycocotools -q
!pip install --upgrade matplotlib seaborn pandas opencv-python tqdm -q

import os
import glob
import random
import shutil
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# ✅ Colab-Specific Setup
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Set random seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

print(f"✅ PyTorch Version: {torch.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

# **1. Download Kaggle Dataset**

In [ ]:
# 🚀 DOWNLOAD KAGGLE MILAIR DATASET
print("\n" + "="*60)
print("📥 DOWNLOADING MILAIR DATASET FROM KAGGLE")
print("="*60)

import kagglehub

# Download latest version
print("📦 Downloading dataset...")
path = kagglehub.dataset_download("nicolassilvanash/milair-dataset")
print(f"✅ Dataset downloaded to: {path}")

# Set dataset paths
DATA_DIR = path
IMAGES_DIR = os.path.join(DATA_DIR, "images")
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "annotations")

# Verify download
print(f"\n📂 Verifying dataset structure:")
print(f"   Data directory: {DATA_DIR}")
print(f"   Images exist: {os.path.exists(IMAGES_DIR)}")
print(f"   Annotations exist: {os.path.exists(ANNOTATIONS_DIR)}")

# Count files
image_files = glob.glob(os.path.join(IMAGES_DIR, "*"))
annotation_files = glob.glob(os.path.join(ANNOTATIONS_DIR, "*.xml"))
print(f"   Number of images: {len(image_files)}")
print(f"   Number of annotations: {len(annotation_files)}")

# Class mapping
CLASS_NAMES = ['ah64', 'chinook', 'cougar', 'f15', 'f16', 'seahawk']
CLASS_MAP = {name: idx for idx, name in enumerate(CLASS_NAMES)}

print(f"\n📊 Classes: {CLASS_NAMES}")

# **2. Explore Dataset**

In [ ]:
def explore_dataset():
    """Explore dataset statistics"""

    xml_files = glob.glob(os.path.join(ANNOTATIONS_DIR, "*.xml"))
    print(f"📊 Analyzing {len(xml_files)} annotations...")

    class_counts = {cls: 0 for cls in CLASS_NAMES}
    image_sizes = []

    for xml_file in tqdm(xml_files[:200], desc="Processing"):  # Limit for speed
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            # Get image size
            size = root.find('size')
            if size is not None:
                width = int(size.find('width').text)
                height = int(size.find('height').text)
                image_sizes.append((width, height))

            # Count objects
            for obj in root.findall('object'):
                name = obj.find('name').text.lower()
                if name in class_counts:
                    class_counts[name] += 1
        except Exception as e:
            continue

    # Display statistics
    print("\n📈 Class Distribution:")
    total_objects = sum(class_counts.values())
    for cls, count in class_counts.items():
        percentage = (count / total_objects * 100) if total_objects > 0 else 0
        print(f"   {cls:10s}: {count:4d} objects ({percentage:.1f}%)")

    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Class distribution
    axes[0].barh(list(class_counts.keys()), list(class_counts.values()))
    axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Count')
    axes[0].grid(True, alpha=0.3)

    # Image size distribution
    if image_sizes:
        widths, heights = zip(*image_sizes)
        axes[1].scatter(widths, heights, alpha=0.5, s=10)
        axes[1].set_title('Image Dimensions', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Width (px)')
        axes[1].set_ylabel('Height (px)')
        axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return len(xml_files)

total_samples = explore_dataset()

# **3. Convert to YOLO Format**

In [ ]:
def convert_to_yolo_format():
    """Convert Pascal VOC to YOLO format"""

    WORKING_DIR = "/content/milair_yolo"
    os.makedirs(WORKING_DIR, exist_ok=True)

    # Create directory structure
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(WORKING_DIR, f'images/{split}'), exist_ok=True)
        os.makedirs(os.path.join(WORKING_DIR, f'labels/{split}'), exist_ok=True)

    # Get all XML files
    xml_files = glob.glob(os.path.join(ANNOTATIONS_DIR, "*.xml"))
    print(f"\n🔄 Converting {len(xml_files)} annotations to YOLO format...")

    # Process files
    processed_files = []

    for xml_file in tqdm(xml_files, desc="Converting"):
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            # Get image filename
            filename = root.find('filename').text
            base_name = os.path.splitext(filename)[0]

            # Find image file
            img_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG']
            img_path = None
            for ext in img_extensions:
                test_path = os.path.join(IMAGES_DIR, base_name + ext)
                if os.path.exists(test_path):
                    img_path = test_path
                    break

            if not img_path:
                continue

            # Get image size
            size = root.find('size')
            if size is None:
                continue

            img_w = int(size.find('width').text)
            img_h = int(size.find('height').text)

            # Extract annotations
            annotations = []
            for obj in root.findall('object'):
                name = obj.find('name').text.lower()
                if name not in CLASS_MAP:
                    continue

                bndbox = obj.find('bndbox')
                if bndbox is None:
                    continue

                try:
                    xmin = float(bndbox.find('xmin').text)
                    ymin = float(bndbox.find('ymin').text)
                    xmax = float(bndbox.find('xmax').text)
                    ymax = float(bndbox.find('ymax').text)

                    # Convert to YOLO format
                    x_center = (xmin + xmax) / 2 / img_w
                    y_center = (ymin + ymax) / 2 / img_h
                    width = (xmax - xmin) / img_w
                    height = (ymax - ymin) / img_h

                    # Validate
                    if (0 <= x_center <= 1 and 0 <= y_center <= 1 and
                        0 < width <= 1 and 0 < height <= 1):
                        cls_id = CLASS_MAP[name]
                        annotations.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                except:
                    continue

            if annotations:
                processed_files.append({
                    'image_path': img_path,
                    'base_name': base_name,
                    'annotations': annotations
                })

        except Exception as e:
            continue

    print(f"✅ Successfully processed: {len(processed_files)}/{len(xml_files)} files")

    # Split dataset
    random.shuffle(processed_files)
    train_ratio, val_ratio = 0.8, 0.1

    train_idx = int(len(processed_files) * train_ratio)
    val_idx = train_idx + int(len(processed_files) * val_ratio)

    train_data = processed_files[:train_idx]
    val_data = processed_files[train_idx:val_idx]
    test_data = processed_files[val_idx:]

    # Save split data
    def save_split(data_list, split_name):
        count = 0
        for item in tqdm(data_list, desc=f"Saving {split_name}"):
            try:
                # Copy image
                img_ext = os.path.splitext(item['image_path'])[1]
                img_dest = os.path.join(WORKING_DIR, f"images/{split_name}",
                                       f"{item['base_name']}{img_ext}")
                shutil.copy2(item['image_path'], img_dest)

                # Save annotations
                label_dest = os.path.join(WORKING_DIR, f"labels/{split_name}",
                                         f"{item['base_name']}.txt")
                with open(label_dest, 'w') as f:
                    f.write('\n'.join(item['annotations']))

                count += 1
            except:
                continue

        return count

    train_count = save_split(train_data, 'train')
    val_count = save_split(val_data, 'val')
    test_count = save_split(test_data, 'test')

    print(f"\n📊 Dataset Split Complete:")
    print(f"   Train: {train_count} images")
    print(f"   Validation: {val_count} images")
    print(f"   Test: {test_count} images")

    # Create data.yaml
    yaml_content = f"""path: {WORKING_DIR}
train: images/train
val: images/val
test: images/test

nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
"""

    yaml_path = os.path.join(WORKING_DIR, "data.yaml")
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"\n✅ Created data.yaml at: {yaml_path}")

    return WORKING_DIR, yaml_path, train_count, val_count, test_count

# Convert dataset
WORKING_DIR, YAML_PATH, train_count, val_count, test_count = convert_to_yolo_format()

# **4. Visualize Sample Annotations**

In [ ]:
def visualize_sample_annotations(num_samples=4):
    """Visualize sample annotations with bounding boxes"""

    train_img_dir = os.path.join(WORKING_DIR, "images/train")
    train_label_dir = os.path.join(WORKING_DIR, "labels/train")

    image_files = glob.glob(os.path.join(train_img_dir, "*"))
    random.shuffle(image_files)

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.ravel()

    colors = plt.cm.tab10(np.linspace(0, 1, len(CLASS_NAMES)))

    for idx, img_path in enumerate(image_files[:num_samples]):
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Load annotations
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(train_label_dir, f"{base_name}.txt")

        h, w = img.shape[:2]

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    cls_id, x_center, y_center, bbox_w, bbox_h = map(float, parts)

                    # Convert to pixel coordinates
                    x_center *= w
                    y_center *= h
                    bbox_w *= w
                    bbox_h *= h

                    x1 = int(x_center - bbox_w / 2)
                    y1 = int(y_center - bbox_h / 2)
                    x2 = int(x_center + bbox_w / 2)
                    y2 = int(y_center + bbox_h / 2)

                    # Draw bounding box
                    color = colors[int(cls_id)]
                    cv2.rectangle(img, (x1, y1), (x2, y2),
                                (int(color[0]*255), int(color[1]*255), int(color[2]*255)), 2)

                    # Add label
                    label = CLASS_NAMES[int(cls_id)]
                    cv2.putText(img, label, (x1, y1-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                               (int(color[0]*255), int(color[1]*255), int(color[2]*255)), 2)

        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'Sample {idx+1}: {base_name}', fontsize=12)

    plt.suptitle('MilAir Dataset - Sample Annotations', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_sample_annotations()

# **5. Colab-Optimized Training**

In [ ]:
# ⚙️ COLAB OPTIMIZED TRAINING CONFIGURATION
print("\n" + "="*60)
print("🚀 COLAB OPTIMIZED TRAINING CONFIGURATION")
print("="*60)

# Clear GPU cache
torch.cuda.empty_cache()

# Initialize model
model = YOLO("yolov8m.pt")  # Medium model for best balance

# Colab-optimized training config
colab_config = {
    'data': YAML_PATH,
    'epochs': 50,           # Reduced for Colab free tier
    'imgsz': 640,
    'batch': 8,             # Reduced for memory constraints
    'workers': 2,           # Reduced workers
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'name': 'milair_kaggle_v1',
    'patience': 20,
    'seed': SEED,

    # Optimization
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,

    # Augmentation (optimized for aerial images)
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 15,          # Aircraft can be at various angles
    'translate': 0.1,
    'scale': 0.2,
    'fliplr': 0.5,
    'mosaic': 0.8,

    # Colab-specific optimizations
    'cache': False,         # Save disk space
    'save_period': 10,
    'exist_ok': True,
    'verbose': True,
    'amp': True,            # Mixed precision training
    'cos_lr': True,         # Cosine learning rate scheduler

    # Save to Google Drive
    'project': '/content/drive/MyDrive/milair_training',
}

print("\n⚙️ Training Configuration:")
print(f"   Model: YOLOv8m")
print(f"   Epochs: {colab_config['epochs']}")
print(f"   Batch Size: {colab_config['batch']}")
print(f"   Image Size: {colab_config['imgsz']}")
print(f"   Classes: {len(CLASS_NAMES)}")
print(f"   Training samples: {train_count}")
print(f"   Save location: {colab_config['project']}")

# **6. Start Training**

In [ ]:
# 🏁 START TRAINING
print("\n" + "="*60)
print("🏁 STARTING TRAINING")
print("="*60)

try:
    # Train the model
    results = model.train(**colab_config)
    print("\n✅ Training completed successfully!")

except Exception as e:
    print(f"\n⚠️ Training error: {e}")

    # Try with simpler configuration
    print("🔄 Trying with simplified configuration...")

    simple_config = colab_config.copy()
    simple_config.update({
        'epochs': 30,
        'batch': 4,
        'imgsz': 416,
        'workers': 1,
        'amp': False,
        'mosaic': 0.0,
        'cache': False,
    })

    results = model.train(**simple_config)
    print("\n✅ Training completed with simplified config!")

# **7. Evaluate Model**

In [ ]:
# 📊 EVALUATE TRAINED MODEL
print("\n" + "="*60)
print("📊 MODEL EVALUATION")
print("="*60)

# Find and load best model
best_model_paths = [
    f"{colab_config['project']}/{colab_config['name']}/weights/best.pt",
    f"/content/drive/MyDrive/milair_training/{colab_config['name']}/weights/best.pt",
    f"/content/runs/detect/{colab_config['name']}/weights/best.pt"
]

best_model = None
for model_path in best_model_paths:
    if os.path.exists(model_path):
        best_model = YOLO(model_path)
        print(f"✅ Loaded best model from: {model_path}")
        break

if best_model is None:
    print("⚠️ Best model not found, using last trained model")
    best_model = model

# Validate on validation set
print("\n🔍 Validating model on validation set...")
metrics = best_model.val(
    data=YAML_PATH,
    split='val',
    conf=0.25,
    iou=0.45,
    device=colab_config['device'],
    verbose=True
)

# Print key metrics
if hasattr(metrics, 'box'):
    print(f"\n📈 Key Metrics:")
    print(f"   mAP50: {metrics.box.map50:.4f}")
    print(f"   mAP50-95: {metrics.box.map:.4f}")

# **8. Visualize Predictions**

In [ ]:
def visualize_predictions_colab(num_images=6):
    """Visualize predictions on test images"""

    # Get test images
    test_img_dir = os.path.join(WORKING_DIR, "images/test")
    if not os.path.exists(test_img_dir):
        test_img_dir = os.path.join(WORKING_DIR, "images/val")

    image_files = glob.glob(os.path.join(test_img_dir, "*"))

    if not image_files:
        print("❌ No test images found")
        return

    # Select random images
    selected_images = random.sample(image_files, min(num_images, len(image_files)))

    # Run predictions
    print(f"🔍 Running predictions on {len(selected_images)} images...")
    results = best_model.predict(
        source=selected_images,
        conf=0.25,
        iou=0.45,
        save=False,
        save_txt=False,
        verbose=False
    )

    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()

    colors = plt.cm.tab10(np.linspace(0, 1, len(CLASS_NAMES)))

    for idx, (result, img_path) in enumerate(zip(results, selected_images)):
        if idx >= 6:
            break

        # Load image
        img = cv2.imread(img_path)
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Draw predictions
        if result.boxes is not None:
            for box in result.boxes.cpu().numpy():
                cls_id = int(box.cls[0])
                conf = box.conf[0]
                bbox = box.xyxy[0].astype(int)

                color = colors[cls_id % len(colors)]
                color_rgb = (int(color[0]*255), int(color[1]*255), int(color[2]*255))

                # Draw bounding box
                cv2.rectangle(img_rgb, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                            color_rgb, 3)

                # Draw label
                label = f"{CLASS_NAMES[cls_id]}: {conf:.2f}"
                cv2.putText(img_rgb, label, (bbox[0], bbox[1]-10),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_rgb, 2)

        axes[idx].imshow(img_rgb)
        axes[idx].axis('off')
        img_name = os.path.basename(img_path)
        detections = len(result.boxes) if result.boxes is not None else 0
        axes[idx].set_title(f'{img_name}\nDetections: {detections}', fontsize=10)

    plt.suptitle('MilAir Dataset - YOLOv8 Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    return results

# Run visualization
prediction_results = visualize_predictions_colab(6)

# **9. Save Results to Google Drive**

In [ ]:
# 💾 SAVE RESULTS TO GOOGLE DRIVE
print("\n" + "="*60)
print("💾 SAVING RESULTS TO GOOGLE DRIVE")
print("="*60)

def save_to_drive():
    """Save all results to Google Drive"""

    # Create results directory in Drive
    drive_results_dir = "/content/drive/MyDrive/milair_results"
    os.makedirs(drive_results_dir, exist_ok=True)

    # Items to save
    items_to_save = [
        (WORKING_DIR, "dataset"),
        (f"{colab_config['project']}/{colab_config['name']}", "training_results"),
    ]

    for source_path, dest_name in items_to_save:
        if os.path.exists(source_path):
            dest_path = os.path.join(drive_results_dir, dest_name)

            # Remove existing if present
            if os.path.exists(dest_path):
                shutil.rmtree(dest_path)

            # Copy
            shutil.copytree(source_path, dest_path)
            print(f"✅ Saved: {dest_name}")

    # Save model separately
    model_dest = os.path.join(drive_results_dir, "best_model.pt")
    if best_model.ckpt_path and os.path.exists(best_model.ckpt_path):
        shutil.copy2(best_model.ckpt_path, model_dest)
        print(f"✅ Saved: best_model.pt")

    # Save predictions
    pred_dir = os.path.join(drive_results_dir, "predictions")
    os.makedirs(pred_dir, exist_ok=True)

    test_img_dir = os.path.join(WORKING_DIR, "images/test")
    if os.path.exists(test_img_dir):
        # Run predictions and save
        test_images = glob.glob(os.path.join(test_img_dir, "*"))[:10]

        for img_path in test_images:
            result = best_model.predict(img_path, save=False, verbose=False)[0]

            # Save annotated image
            img_name = os.path.basename(img_path)
            save_path = os.path.join(pred_dir, f"pred_{img_name}")
            result.save(save_path)

    print(f"\n🎉 All results saved to: {drive_results_dir}")
    print(f"📁 You can access them in your Google Drive")

# Save results
save_to_drive()